In [ ]:
# Definimos los parametros de los agentes trabajadores
MIN_PS_AGENTS = 10          # Numero minimo de resolucion de problemas de los agentes
MAX_PS_AGENTS = 100         # Numero maximo de resolucion de problemas de los agentes
MIN_FRIENDSHIP = 0          # Minimo de amistad entre los agentes
MAX_FRIENDSHIP = 1          # Maximo de amistad entre los agentes
MIN_MIN_ENERGY = 10         # Minima energia de los agentes para trabajar
MAX_MIN_ENERGY = 100        # Maxima energia de los agentes para trabajar
MIN_MOTIVATION = 10         # Motivacion minima para que los trabajadores se sientan motivados

# Definimos los parametros del Project Manager
MIN_MOTIVATION = 20         # Minima motivacion que debe tener el equipo
RISKY = 0.2                 # Cuan arriesgado es el project manager
MIN_WORK_PROB = 0             # Probabilidad minima de que elija trabajar él antes de mandar a un trabajador
MAX_WORK_PROB = 0.2  
MIN_COOPERATION = 0.5       # Probabilidad minima de que elija mandar a cooperar en tareas dificiles, 1 - P seria la probabilidad de que reasigne en vez de cooperar
MAX_COOPERATION = 1
MIN_EXPLORATION = 0         # Probabilidad minima de que tome decisiones poco usuales
MAX_EXPLORATION = 0.2 

# Definimos algunos parametros para las tareas
MIN_DURATION = 5            # Duracion minima de una tarea
MAX_DURATION = 100         
INTERVAL = 1.2              # Porcentaje de tiempo entre el start y el deadline de la tarea
MIN_RESOURCES = 0           # Cantidad minima de recursos que tiene una tarea
MAX_RESOURCES = 10
MIN_DEPENDENCIES = 0        # Cantidad minima de dependencias que tiene una tarea
MAX_DEPENDENCIES = 10
MIN_REWARD = 0              # Recompensa minima que debe tener una tarea
MAX_REWARD = 100 
MIN_DIFFICULTY = 0          # Dificultad minima que debe tener una tarea
MAX_DIFFICULTY = 100
MIN_PROBLEMS_PROB = 0       # Probabilidad minima de una tarea de generar un problema
MAX_PROBLEMS_PROB = 0.5     


### Definimos la simulación

In [1]:
import random
from PMOntologic.PMO import Project
from PMOntologic.Resource import Resource
from Simulation.Simulator import Simulator
from Simulation.Environment import WorkCenter
from Simulation.PMAgent import PMAgent, PMperception
from Simulation.WorkerAgent import WorkerAgent
from Tasks.GeneticAlgorithm.Population import Population
from Tasks.GeneticAlgorithm.Tasks_combination import optimization_function
from Tasks.task import Task

agents = [
    WorkerAgent('trabajador_1', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_2', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_3', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_4', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_5', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_6', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_7', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_8', problem_solving=random.randint(10,70), friendship=random.random()),
    WorkerAgent('trabajador_9', problem_solving=random.randint(10,70), friendship=random.random())
]

resources_id = ['Resource1', 'Resource2', 'Resource3', 'Resource4', 'Resource5', 'Resource6', 'Resource7', 'Resource8', 'Resource9', 'Resource10']
resources_cant = {resource : 0 for resource in resources_id}

# Definamos tareas de modo que no queden dependencias circulares
tasks = []
for i in range(100):
    task = Task(id=i, priority=random.randint(0,5), duration=random.randint(10,60), reward=random.randint(10,100), difficulty=random.randint(10,50), problems_probability=random.uniform(0,0.5))
    cant = random.randint(1,10)
    for _ in range(cant):
        r = random.choice(resources_id)
        amount = random.randint(0,5)
        resource = Resource(r, amount)
        task.resources.append(resource)
        resources_cant[r] += amount 
    tasks.append(task)

for i in range(99):
    if random.random() < 0.2 :
        dep = random.randint(1,4)
        for j in range(dep):
            index = random.randint(i+1, 99)
            tasks[i].dependencies.append(tasks[index])


project = Project(objective='number_of_tasks',tasks=tasks, resources=[Resource(resource,cant*1.2) for resource,cant in resources_cant.items()])

pm = PMAgent(min_motivation_team=20, initial_perception=PMperception(actual_time=0, team_motivation=80), project=project)
pm.know_workers([(agent.id,agent.problem_solving) for agent in agents])

simulator = Simulator(WorkCenter)
simulations = simulator.StartManySimulationsThreading(8,1000,agents,pm,project)    
   



In [3]:
for env in simulations:
    env.save_log_to_csv()  